In [ ]:
# !pip install firebase-admin
# !pip install google-cloud-storage
# !pip install python-dotenv
# !pip install pandas


In [1]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()

True

# Firebase Init

<!-- # Firebase Init -->

In [2]:
service_account_info ={
    "type": os.getenv('FIREBASE_TYPE'),
    "project_id": os.getenv('FIREBASE_PROJECT_ID'),
    "private_key_id": os.getenv('FIREBASE_PRIVATE_KEY_ID'),
    "private_key": os.getenv('FIREBASE_PRIVATE_KEY'),
    "client_email": os.getenv('FIREBASE_CLIENT_EMAIL'),
    "client_id": os.getenv('FIREBASE_CLIENT_ID'),
    "auth_uri": os.getenv('FIREBASE_AUTH_URI'),
    "token_uri": os.getenv('FIREBASE_TOKEN_URI'),
    "auth_provider_x509_cert_url": os.getenv('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
    "client_x509_cert_url": os.getenv('FIREBASE_CLIENT_X509_CERT_URL'),
    "universe_domain": os.getenv('FIREBASE_UNIVERSE_DOMAIN')
  }
  



In [3]:
cred = credentials.Certificate(service_account_info)
firebase_admin.initialize_app(cred,{
     'databaseURL': "https://coffeeshop-app-8d2a6-default-rtdb.firebaseio.com/"
})

<!-- # Upload Data -->

In [8]:
products_collection = db.reference('products')

In [ ]:
df = pd.read_json('products/products.jsonl',lines=True)
df.head(2)

In [ ]:
from google.cloud import storage as g_storage
storage_client = g_storage.Client()
buckets = storage_client.list_buckets()

for bucket in buckets:
        print(bucket.name)

# Upload Data

In [ ]:
def upload_image(bucket, image_path):
    """Uploads a file to the bucket."""
    image_name = image_path.split('/')[-1]
    destination_blob_name = f'product_images/{image_name}'
    blob = bucket.blob(destination_blob_name)

    # generation_match_precondition = 0

    blob.upload_from_filename(image_path)

    # blob.make_public()
    # return blob.public_url
      # Return public URL (no signed URL needed)
    public_url = f"https://storage.googleapis.com/{bucket.name}/{destination_blob_name}"
    print(f"🌐 Public URL: {public_url}")
    return public_url

In [ ]:
bucket = storage_client.bucket("cafe_proj")


for index, row in df.iterrows():
    print(index, row['name'])
    # this is where we want to store the image
    image_folder_path = './products/images/'
    image_path = os.path.join(image_folder_path,row['image_path'])
    # returns the public url for the image 
    image_url = upload_image(bucket,image_path)

    # Product for realtime fire database
    product_data = row.to_dict()
    product_data.pop('image_path')
    product_data['image_url']= image_url
    
    # Add to Firestore
    products_collection.push().set(product_data)

In [ ]:
# def upload_image(bucket, image_path):
#     image_name = image_path.split('/')[-1]
#     blob = bucket.blob(f'product_images/{image_name}')
#     # Upload image
#     blob.upload_from_filename(image_path)
#     # Make the image publicly accessible and get its URL
#     blob.make_public()
#     return blob.public_url

In [4]:
# in YT